# Module 1. Personalize 첫 캠페인 생성하기

이 노트북은 MovieLens 데이터셋에서 수집된 데이터를 기반으로, 영화에 대한 추천 모델을 작성하는 법을 안내합니다. 목표는 특정 사용자를 기반으로 하는 영화를 추천하는 것입니다.

데이터는 MovieLens 프로젝트에서 가져옵니다. 데이터셋 import, 솔루션 생성 및 캠페인 생성에 많은 대기 시간이 소요됩니다. 여러분은 대기 시간 동안 구글링을 통해 데이터 및 잠재적 용도에 대해 자세히 알아볼 수 있습니다.

## Notebook 사용법

코드는 여러 코드 셀들로 구성됩니다. 이 페이지의 상단에 삼각형으로 된 실행 단추를 마우스로 클릭하여 각 셀을 실행하고 다음 셀로 이동할 수 있습니다. 또는 셀에서 키보드 단축키 `Shift + Enter`를 눌러 셀을 실행하고 다음 셀로 이동할 수도 있습니다.

셀이 실행되면 셀이 실행되는 동안 측면에 줄이 * 표시되어 있거나 셀 내의 모든 코드를 예측한 후 실행이 완료된 마지막 셀을 나타내기 위해 숫자로 업데이트됩니다.

아래 지침을 따르고 셀을 실행하여 Amazon Personalize를 시작하세요.

## 라이브러리 임포트

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 핸즈온을 위해서 핵심 데이터 과학 도구인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 데이터 설정

Amazon S3를 통해 Amazon Personalize로 데이터를 가져옵니다. 아래에서는 이 핸즈온을 위해 AWS에서 생성한 버킷을 지정합니다.

아래에서 `bucket` 변수를 업데이트하여 CloudFormation 단계에서 앞서 생성한 값으로 설정해 주세요. 이는 이전 작업의 텍스트 파일에 있어야 합니다. `filename`은 변경할 필요가 없습니다.

### S3 버킷 및 데이터 출력 위치 지정

In [4]:
import sagemaker
bucket = sagemaker.Session().default_bucket()  # replace with the name of your S3 bucket
filename = "movie-lens-100k.csv"

### 훈련 데이터 다운로드, 준비 및 업로드

핸즈온을 수행할 MovieLens 데이터가 아직 로컬로 로드되어 있지 않은 경우 아래 줄을 실행하여 최신 사본을 다운로드하고 신속하게 확인해 보세요.

#### 데이터셋 다운로드 및 탐색

In [5]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

--2020-03-11 07:30:44--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  4.98MB/s    in 0.9s    

2020-03-11 07:30:45 (4.98 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
...,...,...,...,...
99998,13,225,2,882399156
99999,12,203,3,879959583


#### 데이터 준비 및 업로드

이 데이터에는 UserID, ItemID, Rating 및 Timestamp 컬럼이 포함되어 있습니다.<br>
이제 순위가 낮은(low ranking) 항목을 제거하고 모델을 만들기 전에 해당 열을 제거합니다.<br>
완료되면 파일을 새 CSV로 저장한 다음, S3에 업로드합니다.<br>
이 모든 것들은 단순히 아래 코드 셀을 실행하여 수행합니다.

In [6]:
data = data[data['RATING'] > 3.6]                # keep only movies rated 3.6 and above
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### 스키마 생성

Personalize가 데이터를 이해하는 방법의 핵심 구성 요소는 아래 정의 된 스키마(schema)에서 비롯됩니다. 이 설정은 CSV 파일을 통해 제공된 데이터를 요약하는 방법을 Personalize 서비스에 알려줍니다. 열(column)과 유형(type)은 위에서 만든 파일의 내용과 일치합니다.

In [7]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:143656149352:schema/DEMO-schema",
  "ResponseMetadata": {
    "RequestId": "4a035f38-2390-4b64-bb0a-df70f4501102",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 07:31:11 GMT",
      "x-amzn-requestid": "4a035f38-2390-4b64-bb0a-df70f4501102",
      "content-length": "77",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 데이터셋 그룹 생성 및 대기

Personalize에서 가장 큰 단위는 **데이터셋 그룹(Dataset Group)** 이며, 이렇게 하면 데이터, 이벤트 추적기(event tracker), 솔루션(solution) 및 캠페인(campaign)이 분리됩니다. 공통의 데이터 수집을 공유하는 것들을 그룹화합니다. 원하는 경우 아래 그룹명을 자유롭게 변경해 주세요.

#### 데이터셋 그룹 생성

In [8]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-dataset-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:143656149352:dataset-group/DEMO-dataset-group",
  "ResponseMetadata": {
    "RequestId": "d8a80e16-52d4-4204-a61c-d1cf768ac3f8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 07:31:20 GMT",
      "x-amzn-requestid": "d8a80e16-52d4-4204-a61c-d1cf768ac3f8",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 데이터셋 그룹이 활성화 상태가 될 때까지 대기

아래의 모든 항목에서 Dataset Group을 사용하려면 활성화(active)가 되어야 합니다. 아래 셀을 실행하고 DatasetGroup: ACTIVE로 변경될 때까지 기다려 주세요.

In [9]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


#### 데이터셋 생성

그룹 다음으로 생성할 것은 실제 데이터셋입니다. 이 예에서는 1개의 interaction 데이터만 작성합니다. 아래의 코드 셀을 실행하여 데이터셋을 생성해 주세요.

In [10]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "DEMO-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:143656149352:dataset/DEMO-dataset-group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "b73aeff0-cc3a-4d26-807d-4c89d34f2008",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 07:36:35 GMT",
      "x-amzn-requestid": "b73aeff0-cc3a-4d26-807d-4c89d34f2008",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### S3 버킷에 정책 부여

Amazon Personalize는 앞서 생성한 S3 버킷의 내용을 읽을 수 있어야 합니다. 아래 코드 셀로 S3 버킷 접근 정책(policy)을 부여합니다.

In [17]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '95907AEF14B894E4',
  'HostId': 'N2G3YA5qPMyl2tHyOvDkTTuLJexfxQEN1pRfY2irOfM4ghuo/01k9XBAc0ZLiI3+E8MQupgSEGY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'N2G3YA5qPMyl2tHyOvDkTTuLJexfxQEN1pRfY2irOfM4ghuo/01k9XBAc0ZLiI3+E8MQupgSEGY=',
   'x-amz-request-id': '95907AEF14B894E4',
   'date': 'Wed, 11 Mar 2020 07:42:50 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

#### Personalize 역할 생성

또한, Amazon Personalize는 특정 작업들을 실행할 권한을 갖기 위해, AWS에서 역할을 맡을 수 있는 기능이 필요합니다. 

In [18]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::143656149352:role/PersonalizeRoleDemo


## 데이터 Import

이전에는 정보를 저장하기 위해 데이터셋 그룹 및 데이터셋을 생성했으므로, 
이제는 모델 구축을 위해 S3에서 Amazon Personalize로 데이터를 로드하는 import job을 실행합니다.

#### 데이터셋 Import Job 생성

In [19]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import1",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:143656149352:dataset-import-job/personalize-demo-import1",
  "ResponseMetadata": {
    "RequestId": "f9919417-f901-483d-bc59-77b6e4abe735",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 07:45:09 GMT",
      "x-amzn-requestid": "f9919417-f901-483d-bc59-77b6e4abe735",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 데이터셋 Import job이 활성화 상태가 될 때까지 대기

Import job이 완료되기까지 시간이 걸립니다. 아래 코드 셀의 출력 결과가 DatasetImportJob: ACTIVE가 될 때까지 기다려 주세요.

In [20]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

###  레시피 선택

In [21]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # 데모를 위해 aws-hrnn 레시피를 선택합니다.
list_recipes_response

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 65000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

### 솔루션 생성 및 대기

먼저 API를 사용하여 솔루션을 생성 후, 버전을 생성합니다.

#### 솔루션 생성

솔루션을 생성합니다. 참고로, 솔루션은 레이블 유형의 식별자일 뿐이며 실제로 훈련된 모델인 버전 생성이 필요합니다.

In [22]:
create_solution_response = personalize.create_solution(
    name = "personalize-poc-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:143656149352:solution/DEMO-solution",
  "ResponseMetadata": {
    "RequestId": "5d7ea42b-4196-4383-8c54-c57c2586778d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 08:29:56 GMT",
      "x-amzn-requestid": "5d7ea42b-4196-4383-8c54-c57c2586778d",
      "content-length": "83",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 솔루션 버전 생성

이 프로세스의 완료는 실제로 25분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

In [23]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:143656149352:solution/DEMO-solution/eef649a4",
  "ResponseMetadata": {
    "RequestId": "67412746-5c6f-430e-b840-89b5c7b14e47",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 08:29:59 GMT",
      "x-amzn-requestid": "67412746-5c6f-430e-b840-89b5c7b14e47",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 솔루션 버전이 활성화 상태가 될 때까지 대기

아래 코드 셀의 실행이 완료되까지는 적어도 20분이 소요됩니다. 시작되고 진행 중 알림(CREATE IN_PROGRESS)이 표시되면 휴식을 취하고 커피를 마시기 좋은 시간입니다.

In [24]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

#### 솔루션 평가 지표 얻기

본 노트북의 시작 시점으로부터 최대 약 1시간 후에 솔루션을 검토할 수 있어야 합니다. 솔루션 버전 생성이 진행되는 동안 다양한 알고리즘들을 확인해 보세요. 솔루션을 캠페인에 배포할 때에도 추가 시간이 소요되므로, 섹션을 2개로 나눌 수 있습니다. 또한 데이터가 시스템에 어떻게 공급되었는지, 어떤 종류의 결과를 기대할 수 있는지에 대한 대안을 논의하기에도 좋은 시기입니다.

솔루션 및 버전이 존재하므로 성능을 판단하기 위한 평가 지표를 얻을 수 있습니다. 본 핸즈온의 데이터는 데모을 위한 스몰 데이터이므로 지표가 좋지 않지만, 더 많은 데이터를 사용하면 개선된 지표를 확인할 수 있습니다.

#### [역자 추가 Tip]

전체 데이터를 모두 훈련에 사용하지 않고 데이터의 일부(예: 약 10%)를 추가 검증을 위한 "미래 데이터"로 홀드-아웃(hold-out) 검증셋으로 지정하여
홀드-아웃 데이터셋에 대해 모델 성능을 별도로 평가할 수도 있습니다.

In [25]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:143656149352:solution/DEMO-solution/eef649a4",
  "metrics": {
    "coverage": 0.3156,
    "mean_reciprocal_rank_at_25": 0.1281,
    "normalized_discounted_cumulative_gain_at_10": 0.1309,
    "normalized_discounted_cumulative_gain_at_25": 0.1917,
    "normalized_discounted_cumulative_gain_at_5": 0.1051,
    "precision_at_10": 0.0337,
    "precision_at_25": 0.0288,
    "precision_at_5": 0.0404
  },
  "ResponseMetadata": {
    "RequestId": "7b60dcc4-a1aa-4e23-b9ef-f0734c1e6657",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 09:13:06 GMT",
      "x-amzn-requestid": "7b60dcc4-a1aa-4e23-b9ef-f0734c1e6657",
      "content-length": "397",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 캠페인 생성 및 대기

작동하는 솔루션 버전을 보유하고 있으므로, 이제 애플리케이션과 함께 사용할 캠페인을 작성해야 합니다. 캠페인은 단순히 모델의 호스팅된 사본입니다. 물론 인프라가 프로비저닝되기까지의 시간이 소요됩니다.

#### 캠페인 생성

In [26]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:143656149352:campaign/DEMO-campaign",
  "ResponseMetadata": {
    "RequestId": "919e33de-5d88-49a5-a42e-4ee8922d3b8a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Mar 2020 12:52:48 GMT",
      "x-amzn-requestid": "919e33de-5d88-49a5-a42e-4ee8922d3b8a",
      "content-length": "83",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 캠페인이 활성화 상태가 될 때까지 대기

In [27]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


## 샘플 추천 결과 얻기

캠페인이 활성화되면 추천 결과를 받을 수 있습니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음, ID 대신 추천을 위해 영화 정보를 표시하는 몇 가지 헬퍼 함수를 만듭니다.

In [28]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

USER: 486


In [29]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    return items.iloc[movie_id]['TITLE']

#### GetRecommendations 호출

아래 코드 셀을 실행하면 특정 사용자에 대한 추천 사항이 표시되고 추천 영화 목록이 반환됩니다.

In [30]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
recommendations_df

Recommendations for user:  486


,OriginalRecs
0,Richard III (1995)
1,Babe (1995)
2,Sleeper (1973)
3,Restoration (1995)
4,Snow White and the Seven Dwarfs (1937)
5,Supercop (1992)
6,"Godfather, The (1972)"
7,Raising Arizona (1987)
8,Local Hero (1983)
9,True Crime (1995)


## 리뷰

위의 코드를 사용하여 이전 사용자 행동을 기반으로 영화 추천을 생성하기 위한 딥러닝 모델을 성공적으로 훈련했습니다. 이 데이터를 사용할 수 있는 다른 유형의 문제들과 이러한 추천 항목들을 제공하기 위해 이와 같은 시스템을 구축하는 것이 어떤 모습일지 생각해 보세요.

이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`2.View_Campaign_And_Interactions.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [32]:
%store campaign_arn
%store dataset_group_arn
%store solution_version_arn
%store solution_arn
%store dataset_arn
%store campaign_arn
%store schema_arn
%store bucket
%store filename
%store role_name
%store recommendations_df
%store user_id

Stored 'campaign_arn' (str)
Stored 'dataset_group_arn' (str)
Stored 'solution_version_arn' (str)
Stored 'solution_arn' (str)
Stored 'dataset_arn' (str)
Stored 'campaign_arn' (str)
Stored 'schema_arn' (str)
Stored 'bucket' (str)
Stored 'filename' (str)
Stored 'role_name' (str)
Stored 'recommendations_df' (DataFrame)
Stored 'user_id' (int64)
